In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

In [25]:
def acteur_realisateur_name(liste_films, df_name_basics, df_acteurs, df_realisateurs):
    
    df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession'], axis = 1)
    df_name_basics = df_name_basics.rename({'knownForTitles' : 'tconst'}, axis = 1)
    df_name_basics['tconst'] = df_name_basics['tconst'].apply(lambda x : x.split(','))
    df_name_basics['nconst'] = df_name_basics['nconst'].astype(str)
    df_name_basics['nconst'] = df_name_basics['nconst'].apply(lambda x : x.replace(' ', ''))
    
    for element in [df_acteurs, df_realisateurs]:

        element = element.drop(['ordering', 'category', 'job', 'characters'], axis = 1)
        element['nconst'] = element['nconst'] + ', '
        element = element.groupby('tconst').sum()
        element = element.reset_index()

        element = pd.merge(liste_films,
                           element,
                           left_on = 'film_id_out_KNN',
                           right_on = 'tconst',
                           how = 'left')

        element = element.drop('film_id_out_KNN', axis = 1)
        element['nconst'] = element['nconst'].astype(str)
        element['nconst'] = element['nconst'].apply(lambda x : x.split(','))
        element = element.explode('nconst')
        element['tconst'] = element['tconst'] + ','
        element = element.groupby('nconst').sum()
        element = element.reset_index()
        element = element[~(element['nconst']== ' ')]
        element['nconst'] = element['nconst'].astype(str)
        element['tconst'] = element['tconst'].astype(str)
        element['nconst'] = element['nconst'].apply(lambda x : x.replace(' ', ''))
        element['tconst'] = element['tconst'].apply(lambda x : x.replace(' ', ''))

    df_acteurs_name = pd.merge(df_acteurs,
                               df_name_basics,
                               on = 'nconst',
                               how = 'left')
    
    df_realisateurs_name = pd.merge(df_realisateurs,
                                    df_name_basics,
                                    on = 'nconst',
                                    how = 'left')
    
    for element2 in [df_acteurs_name, df_realisateurs_name]:
        element2['tconst_y'] = element2['tconst_y'].astype(str)
        element2['tconst'] = element2['tconst_x'] + element2['tconst_y']
        element2 = element2[['nconst', 'primaryName', 'tconst']]
        element2['tconst'] = element2['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
        element2['tconst'] = element2['tconst'].apply(lambda x : x.split(','))
        element2 = element2.rename({'tconst': 'knownForTitles'}, axis = 1)

    df_realisateurs_name = df_realisateurs_name.rename({'nconst': 'realisateurs'}, axis = 1)
    df_acteurs_name = df_acteurs_name.rename({'nconst': 'acteur'}, axis = 1)

    return df_name_basics, df_acteurs_name, df_realisateurs_name

IMPORT DES BASES

In [3]:
df_films = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')

In [4]:
liste_films = pd.DataFrame(df_films['film_id_out_KNN'])

In [6]:
liste_films = liste_films.drop_duplicates()

In [7]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

TABLE ACTEURS

In [8]:
df_acteurs = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self')]

TABLE REALISATEURS

In [9]:
df_realisateurs = df_title_principals[df_title_principals['category'] == 'director']

In [10]:
df_title_principals = 0

TITLE CREW

In [11]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [12]:
df_title_crew = df_title_crew.drop('writers', axis = 1)

TITLE CREW

In [13]:
df_title_crew = pd.merge(liste_films,
                      df_title_crew,
                      left_on = 'film_id_out_KNN',
                      right_on = 'tconst',
                      how = 'left')

In [14]:
df_title_crew = df_title_crew.drop('film_id_out_KNN', axis = 1)

In [15]:
df_title_crew = df_title_crew.rename({'directors': 'nconst'}, axis = 1)

In [16]:
df_title_crew['nconst'] = df_title_crew['nconst'].astype(str)

In [17]:
df_title_crew['nconst'] = df_title_crew['nconst'].apply(lambda x : x.split(","))

In [18]:
df_title_crew = df_title_crew.explode('nconst')

In [19]:
df_title_crew = df_title_crew[~(df_title_crew['nconst'] == " ")]

In [20]:
df_title_crew['tconst'] = df_title_crew['tconst'] + ', '

In [21]:
df_title_crew = df_title_crew.groupby('nconst').sum()

In [22]:
df_title_crew = df_title_crew.reset_index()

In [ ]:
df_title_crew

TITLE CREW OK

AJOUT DES NAMES

In [23]:
df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [ ]:
df_name_basics, df_acteurs_name, df_realisateurs_name = acteur_realisateur_name(liste_films, df_name_basics, df_acteurs, df_realisateurs)

In [ ]:
df_acteurs_name

In [196]:
df_acteurs_old = pd.read_csv('BD/P2_G5_acteurs.csv.gz', compression = 'gzip')

In [ ]:
df_acteurs_old

In [83]:
# df_name_basics['nconst'] = df_name_basics['nconst'] + ', '
# df_name_basics = df_name_basics.explode('tconst')
# films = df_films['film_id_out_KNN'].to_list()
# df_name_basics['films'] = df_name_basics['tconst'].apply(lambda x : x in films)
# df_name_basics = df_name_basics[df_name_basics['films'] == True]
# df_name_basics = df_name_basics.drop('films', axis = 1)

EXPORT

In [ ]:
df_acteurs_name.to_csv('P2_G5_acteurs.csv.gz', index=False, compression='gzip')
df_realisateurs_name.to_csv('P2_G5_realisateurs.csv.gz', index=False, compression='gzip')